In [89]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [90]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [91]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-manual").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

101 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'std': 5.91778405040492

In [92]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

101 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,direct/format-few.txt,cq.txt,...,5.0,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.0,28.0
1,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,4c3a8040449c9603bec5032b54162c41f53c442b,bifid-snow,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,430.5,1068.0,65.0,799.169231,1610.523271,159.0,269.0,342.0,440.0,8193.0
2,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,a5354126b8078a8578850a29c2b8f2a34a1ac2d3,ovoid-puds,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,359.0,1187.0,63.0,521.571429,1005.241196,159.0,259.0,344.0,448.0,8065.0
3,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,80698cde292df8ac95412aa9953826f94f6bba76,piano-mean,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,355.5,524.0,73.0,380.246575,212.387276,123.0,255.0,324.0,438.0,1308.0
4,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,177129695d4899f693b218a96e449e6a2dbf2675,spumy-sign,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,354.5,691.0,69.0,711.971014,1534.491360,159.0,255.0,321.0,444.0,8193.0


In [93]:
if 'params.qa.top_p' not in df.columns:
    df['params.qa.top_p'] = None

In [94]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [95]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols, keep='last', inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

96 experiments after preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.top_p
1,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,4c3a8040449c9603bec5032b54162c41f53c442b,bifid-snow,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,1068.0,65.0,799.169231,1610.523271,159.0,269.0,342.0,440.0,8193.0,None
2,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,a5354126b8078a8578850a29c2b8f2a34a1ac2d3,ovoid-puds,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,1187.0,63.0,521.571429,1005.241196,159.0,259.0,344.0,448.0,8065.0,None
3,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,80698cde292df8ac95412aa9953826f94f6bba76,piano-mean,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,524.0,73.0,380.246575,212.387276,123.0,255.0,324.0,438.0,1308.0,None
4,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,177129695d4899f693b218a96e449e6a2dbf2675,spumy-sign,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,691.0,69.0,711.971014,1534.491360,159.0,255.0,321.0,444.0,8193.0,None
5,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,50f737766dbdddf92ac692d95a8aff45e9468c32,forte-haet,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,932.0,73.0,437.369863,814.629679,123.0,227.0,317.0,436.0,7161.0,None


In [96]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [97]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [98]:
# make temperature 1 decimal
df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [99]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['train']

- params.qa.model: ['deepseek-r1-llama-8b', 'deepseek-r1-llama-70b', 'llama-3-8b', 'llama-3.3-70b', 'qwen-2.5-32b']

- params.qa.temperature: [0.0, 0.5]

- params.qa.system_prompt: ['empty.txt', 'ccot/format-thought.txt', 'cot/format-thought.txt', 'direct/format-few.txt']

- params.qa.user_prompt_template: ['icq-format.txt', 'cq.txt']

- params.qa.few_shot_examples: ['manual/reasoning.json', 'manual/ccot.json', 'manual/cot.json', 'manual/direct.json']

- params.qa.n_shot: [0, 1]

- params.qa.n_sc: [1]

- params.run: [3, 1, 2]

- params.qa.top_p: [None]



In [100]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [101]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [102]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [103]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.split": ["train"],
    "params.qa.temperature": [
        0.0,
        0.5,
    ],
    "params.qa.top_p": [
        None,
        0.95,
    ],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_sc": [
        1,
    ],
    "params.qa.n_shot": [
        0,
        # 1,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [104]:
varying_params_list = [
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["direct/format-few.txt"],
        "params.qa.few_shot_examples": ["manual/direct.json"],
    },
    # {
    #     "params.qa.model": [
    #         # "llama-3-8b",
    #         "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cot/format-thought.txt"],
    #     "params.qa.few_shot_examples": ["manual/cot.json"],
    # },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["ccot/format-thought.txt"],
        "params.qa.few_shot_examples": ["manual/ccot.json"],
    },
    # {
    #     "params.qa.model": [
    #         "deepseek-r1-llama-8b",
    #         "deepseek-r1-llama-70b",
    #     ],
    #     "params.qa.system_prompt": ["empty.txt"],
    #     "params.qa.user_prompt_template": ["icq-format.txt"],
    #     "params.qa.few_shot_examples": ["manual/reasoning.json"],
    # },
]

In [105]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

24 experiment configurations
['params.dataset.path', 'params.dataset.split', 'params.qa.temperature', 'params.qa.top_p', 'params.qa.user_prompt_template', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.system_prompt', 'params.qa.few_shot_examples']


In [106]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 96


In [107]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

12 missing configurations


In [108]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if isinstance(arg_value, str) and '[' in arg_value:
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [109]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [110]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.top_p,params.qa.technique,params.qa.instruction
1,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,4c3a8040449c9603bec5032b54162c41f53c442b,bifid-snow,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,799.169231,1610.523271,159.0,269.00,342.0,440.00,8193.0,None,REASONING,empty:icq-format:manual/reasoning
2,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,a5354126b8078a8578850a29c2b8f2a34a1ac2d3,ovoid-puds,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,521.571429,1005.241196,159.0,259.00,344.0,448.00,8065.0,None,REASONING,empty:icq-format:manual/reasoning
3,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,80698cde292df8ac95412aa9953826f94f6bba76,piano-mean,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,380.246575,212.387276,123.0,255.00,324.0,438.00,1308.0,None,REASONING,empty:icq-format:manual/reasoning
4,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,177129695d4899f693b218a96e449e6a2dbf2675,spumy-sign,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,711.971014,1534.491360,159.0,255.00,321.0,444.00,8193.0,None,REASONING,empty:icq-format:manual/reasoning
5,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,50f737766dbdddf92ac692d95a8aff45e9468c32,forte-haet,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,437.369863,814.629679,123.0,227.00,317.0,436.00,7161.0,None,REASONING,empty:icq-format:manual/reasoning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,b018a25b61d3cc0ae6e1e34779cfc29b7f86eca1,bc661b06e1dc58ee791f1bf9e85a7ecc42052b53,pawky-flex,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.5,direct/format-few.txt,cq.txt,...,9.452381,7.877731,3.0,4.00,5.0,12.75,35.0,None,DIRECT,direct/format-few:cq:manual/direct
97,b018a25b61d3cc0ae6e1e34779cfc29b7f86eca1,c7b85a1dc9e0beff3e496eb776b44aee2df745e3,sooty-hoot,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.5,ccot/format-thought.txt,cq.txt,...,28.130435,10.444794,14.0,23.00,26.0,30.00,59.0,None,CCOT,ccot/format-thought:cq:manual/ccot
98,b018a25b61d3cc0ae6e1e34779cfc29b7f86eca1,0fbdab0c4ea5b6e0c235f75dbcdee32efa87fdf9,woven-erns,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.5,ccot/format-thought.txt,cq.txt,...,41.684211,16.846052,16.0,30.25,38.5,53.25,78.0,None,CCOT,ccot/format-thought:cq:manual/ccot
99,b018a25b61d3cc0ae6e1e34779cfc29b7f86eca1,175a39884841457f9b39d8ba3c287407a108355e,purer-gram,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.5,ccot/format-thought.txt,cq.txt,...,39.589744,24.162518,12.0,25.50,31.0,46.00,138.0,None,CCOT,ccot/format-thought:cq:manual/ccot
